<h1> Reem Khaled Elsayed Aboushama </h1>  </br>  <h2>Data Mining and Business Intelligence</h2> </br> <h2> CSE 382 </h2>

In [1]:
import pandas as pd
import numpy as np

<h2>Reading the data<h2>

In [ ]:
data = pd.read_csv(r"creditcard.csv")
